# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from os import listdir as LD, makedirs as MDs
from os.path import join as PJ, basename as PBN, dirname as PDN, exists as PE
import pandas as pd
from datetime import datetime as DT

In [2]:
# from imod import msw
# from imod import mf6
# import primod

In [3]:
import WS_Mdl.utils as U
import WS_Mdl.utils_imod as UIM
import WS_Mdl.calcs as C

In [4]:
import importlib as IL
IL.reload(U)
IL.reload(UIM)

<module 'WS_Mdl.utils_imod' from 'C:\\Users\\Karam014\\OneDrive - Universiteit Utrecht\\WS_Mdl\\code\\WS_Mdl\\utils_imod.py'>

In [5]:
# Import sfrmaker and other necessary packages for SFR network creation
import sfrmaker as sfr
import geopandas as gpd
import numpy as np

In [6]:
import fiona

In [7]:
sfr.__version__

'0.11.3'

In [8]:
from WS_Mdl.utils import bold, style_reset

# Options

In [9]:
Pa_GPkg_1ry = r'C:\OD\WS_Mdl\models\NBr\PrP\SFR\BrabantseDelta\WBD_SW_NW_1ry.gpkg' 
Pa_GPkg = r"C:\OD\WS_Mdl\models\NBr\PrP\SFR\BrabantseDelta\acceptatiedatabase.gdb"
detailed = 'hydroobject'
primary = 'LEGGER_VASTGESTELD_WATERLOOP_CATEGORIE_A'

In [10]:
MdlN = 'NBr33'

In [11]:
U.set_verbose(False)

In [12]:
# Load paths and variables from PRJ & INI
d_Pa = U.get_MdlN_Pa(MdlN)
Pa_PRJ = d_Pa['PRJ']
Dir_PRJ = PDN(Pa_PRJ)
d_INI = U.INI_to_d(d_Pa['INI'])
Xmin, Ymin, Xmax, Ymax = [float(i) for i in d_INI['WINDOW'].split(',')]
SP_date_1st, SP_date_last = [DT.strftime(DT.strptime(d_INI[f'{i}'], '%Y%m%d'), '%Y-%m-%d') for i in ['SDATE', 'EDATE']]
dx = dy = float(d_INI['CELLSIZE'])

# 1ry SW Nw PrP

In [ ]:
GDF_all = gpd.read_file(Pa_GPkg_1ry)

In [91]:
GDF_all

,CODE,WS_STATUS_L,WS_LEGGERCATEGORIE_L,WS_LEGGERBRON_L,DATUM_VASTGESTELD,LEGGER_KENMERK,WS_ONDERHOUDSPLICHTIGE_L,WS_ONDERHOUDSPLICHT_L,WS_BODEMBREEDTE_L,WS_BH_BOVENSTROOMS_L,...,WS_OMSCHRIJVING_TRAJECT_L,WS_LEGGERVERWIJZING_L,WS_HYPERLINK_PRO_OPW_L,WS_HYPERLINK_PRO_VW_L,WS_LEGGERBRON_OMW_L,WS_LEGGERBRON_WAW_L,WS_LEGGERBRON_COS_L,WS_HYPERLINK_BESLUIT_L,SHAPE_Length,geometry
0,OVK12326,"Vastgesteld, Besluit geverifieerd, ingemeten",Oppervlaktewaterlichamen,"11UT009138, kernregistratie",2021-03-30 00:00:00+00:00,"Waterschapsblad 2021, 4434",None,None,0.50,18.45,...,None,Niet van toepassing,None,None,None,None,None,None,103.786374,"MULTILINESTRING ((82178.821 382139.593, 82180...."
1,OVK21780,"Vastgesteld, Besluit geverifieerd, ingemeten",Oppervlaktewaterlichamen,Oude legger de gewijzigde Cruijslandpolders 19...,2021-03-30 00:00:00+00:00,"Waterschapsblad 2021, 4434",None,None,1.50,-1.64,...,None,Niet van toepassing,None,None,None,None,None,None,159.245366,"MULTILINESTRING ((87960.311 397958.808, 87960...."
2,OVK02233,"Vastgesteld, Ligging gewijzigd, tabel ongewijzigd",Oppervlaktewaterlichamen,None,2021-03-30 00:00:00+00:00,"Waterschapsblad 2021, 4434",None,None,0.60,2.78,...,None,Niet van toepassing,None,None,None,None,None,None,211.357944,"MULTILINESTRING ((111124.488 396263.763, 11113..."
3,OVK07620,"Vastgesteld, Ligging gewijzigd, tabel ongewijzigd",Oppervlaktewaterlichamen,None,2021-03-30 00:00:00+00:00,"Waterschapsblad 2021, 4434",None,None,0.90,-2.19,...,None,Niet van toepassing,None,None,None,None,None,None,565.948226,"MULTILINESTRING ((104001.385 404319.356, 10400..."
4,OVK05547,"Vastgesteld, Besluit geverifieerd, ingemeten",Oppervlaktewaterlichamen,"Oude legger De polders van Nieuw Vosmeer 1971,...",2021-03-30 00:00:00+00:00,"Waterschapsblad 2021, 4434",None,None,1.77,-1.56,...,None,Niet van toepassing,None,None,None,None,None,None,1415.946440,"MULTILINESTRING ((75046.141 401865.437, 75045...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12859,OVK03414,Vigerende legger,Oppervlaktewaterlichamen,None,2024-11-21 00:00:00+00:00,"Waterschapsblad 2024, 25527",Gemeente Waalwijk,None,0.50,-0.06,...,None,Niet van toepassing,None,None,14IN032214,None,None,https://zoek.officielebekendmakingen.nl/wsb-20...,230.941879,"MULTILINESTRING ((127154.852 413725.971, 12715..."
12860,OVK08022,Vigerende legger,Oppervlaktewaterlichamen,None,2024-11-21 00:00:00+00:00,"Waterschapsblad 2024, 25527",None,None,0.92,-0.98,...,None,Niet van toepassing,None,None,448580,None,None,https://zoek.officielebekendmakingen.nl/wsb-20...,562.094599,"MULTILINESTRING ((106839.183 406980.574, 10685..."
12861,OVK21240,Vigerende legger,Oppervlaktewaterlichamen,None,2024-11-21 00:00:00+00:00,"Waterschapsblad 2024, 25527",None,None,1.00,1.50,...,None,Niet van toepassing,None,None,757483,None,waterloop was op de legger cat-C tijdens vergu...,https://zoek.officielebekendmakingen.nl/wsb-20...,405.070769,"MULTILINESTRING ((98685.008 394485.756, 98680...."
12862,OVK07617,Vigerende legger,Oppervlaktewaterlichamen,None,2024-11-21 00:00:00+00:00,"Waterschapsblad 2024, 25527",None,None,0.80,-1.90,...,None,Niet van toepassing,None,None,IP800380 verbetering RWK,None,None,https://zoek.officielebekendmakingen.nl/wsb-20...,512.689618,"MULTILINESTRING ((102983.931 404344.646, 10300..."


## Calculate routing

## Extract coordinates and identify downstream segments.

### Create Xend and Yend columns

In [92]:
from shapely.geometry import MultiLineString

In [93]:
GDF_all['Xstart'] = GDF_all['geometry'].apply(lambda x: x.geoms[0].coords[0][0]) # Access X coorddinate of first point in first linestring
GDF_all['Ystart'] = GDF_all['geometry'].apply(lambda x: x.geoms[0].coords[0][1])
GDF_all['Xend'] = GDF_all['geometry'].apply(lambda x: x.geoms[0].coords[-1][0])
GDF_all['Yend'] = GDF_all['geometry'].apply(lambda x: x.geoms[0].coords[-1][1])

In [95]:
print("✓ Start and end coordinates extracted.")
l_coord_cols = ['CODE', 'Xstart', 'Ystart', 'Xend', 'Yend']
GDF_all[l_coord_cols].head()

✓ Start and end coordinates extracted.


,CODE,Xstart,Ystart,Xend,Yend
0,OVK12326,82178.8210,382139.5930,82211.0763,382230.8210
1,OVK21780,87960.3111,397958.8084,87987.8930,397803.1720
2,OVK02233,111124.4879,396263.7630,111286.6935,396369.1693
3,OVK07620,104001.3850,404319.3560,104532.9200,404493.5701
4,OVK05547,75046.1407,401865.4367,73901.1920,401618.4030


### Limit X, Y to model area

In [137]:
GDF = GDF_all[  ( (GDF_all['Xstart'].between(Xmin, Xmax, inclusive='both') | GDF_all['Xend'].between(Xmin, Xmax, inclusive='both') ) &
              (GDF_all['Ystart'].between(Ymin, Ymax, inclusive='both') | GDF_all['Yend'].between(Ymin, Ymax, inclusive='both') ) ) ]

In [138]:
# Create a lookup dictionary from start coordinates to CODE
coord_to_id = {(row.Xstart, row.Ystart): row.CODE for row in GDF.itertuples()}

print(f"✓ Lookup dictionary created with {bold}{len(coord_to_id)}{style_reset} entries.")

✓ Lookup dictionary created with 583 entries.


The number of features selected in QGIS (within model area) is 562, which suprisingly is bigger than 556. We'll continue without investigating further for now.

### Identify downstream

In [139]:
# Function to find the downstream ID
def get_toid(row):
    end_coords = (row.Xend, row.Yend)
    return coord_to_id.get(end_coords, 0)

# Apply the function to create the 'toid' column
GDF['toid'] = GDF.apply(get_toid, axis=1)

print("✓ 'toid' column calculated.")

✓ 'toid' column calculated.


In [140]:
print(f'{round(GDF['toid'].value_counts().max()/GDF.shape[0]*100, 2)} % of toids are 0 (i.e. no start coordinates match the end coordinates of the current node).')

9.48 % of toids are 0 (i.e. no start coordinates match the end coordinates of the current node).


The percentage is bigger than expected. Let's investigate.

## Investigate segments that failed to connect

### Check out number of matches/no matches

In [141]:
GDF['toid_match'] = GDF['toid'].isin(GDF['CODE'])
GDF['toid'].value_counts()

toid
0           56
OVK03070     3
OVK11688     3
OVK02834     2
OVK01661     2
            ..
OVK02106     1
OVK02327     1
OVK01633     1
OVK01349     1
OVK01433     1
Name: count, Length: 451, dtype: int64

It makes sense that most nodes have just a few upstream nodes. 55 nodes are not connected.

In [142]:
GDF['toid_match'].value_counts()

toid_match
True     535
False     56
Name: count, dtype: int64

### Calculate min distance from start to any reach's end and investigate no matches.

In [143]:
GDF['min_Dist'] = 0.0
GDF.loc[GDF['toid'] == 0, 'min_Dist'] = GDF.loc[GDF['toid'] == 0].apply(lambda row: C.c_Dist(row['Xend'], row['Yend'], GDF['Xstart'], GDF['Ystart']).min(), axis=1)

In [144]:
N_total_no_match = (GDF['toid'] == 0).sum()

In [145]:
GDF.loc[GDF['toid_match']==False,'min_Dist'].describe()

count     56.000000
mean      50.930158
std      119.328808
min        0.000100
25%        0.000100
50%        0.000241
75%       10.010838
max      564.173123
Name: min_Dist, dtype: float64

In [ ]:
l_Vals = [0.001, 0.1, 1, 10, 100, 1000, 10000]

print(f'Out of the {N_total_no_match} segments that do not match:')

N_below_Prv, Val_Prv = 0, 0
for v in l_Vals:
    N_below = (GDF.loc[GDF['toid_match']==False,'min_Dist'] <= v).sum()
    P_below = round(N_below / N_total_no_match * 100, 2)
    
    sample_gdf = GDF.loc[(GDF['min_Dist'] > Val_Prv) & (GDF['min_Dist'] <= v), ['CODE', 'min_Dist']].sort_values(by='min_Dist')
    sample_gdf['Code:min_Dist'] = sample_gdf.apply(lambda row: f"{row['CODE']}: {row['min_Dist']:8.4f}", axis=1)
    sample_gdf = sample_gdf['Code:min_Dist']
    example_nodes = sample_gdf.iloc[:].tolist()

    print(f'-{Val_Prv:6} < min_Dist <= {v:5} |N: {N_below:6} (+ {(N_below - N_below_Prv):4}) ({round(P_below,1):5} %) | Codes: {example_nodes}\n')

    N_below_Prv, Val_Prv = N_below, v

Out of the 56 segments that do not match:
-     0 < min_Dist <= 0.001 |N:     37 (+   37) ( 66.1 %) | Example nodes: ['OVK02919:   0.0001', 'OVK02936:   0.0001', 'OVK02055:   0.0001', 'OVK02089:   0.0001', 'OVK01666:   0.0001', 'OVK20480:   0.0001', 'OVK20466:   0.0001', 'OVK02235:   0.0001', 'OVK00961:   0.0001', 'OVK20990:   0.0001', 'OVK01440:   0.0001', 'OVK00937:   0.0001', 'OVK01445:   0.0001', 'OVK00976:   0.0001', 'OVK01793:   0.0001', 'OVK01694:   0.0001', 'OVK02909:   0.0001', 'OVK00907:   0.0001', 'OVK01716:   0.0001', 'OVK01492:   0.0001', 'OVK02214:   0.0001', 'OVK00960:   0.0001', 'OVK00965:   0.0001', 'OVK01537:   0.0001', 'OVK02101:   0.0001', 'OVK01675:   0.0001', 'OVK02326:   0.0001', 'OVK01803:   0.0002', 'OVK20449:   0.0003', 'OVK02098:   0.0003', 'OVK22007:   0.0003', 'OVK01674:   0.0004', 'OVK00975:   0.0005', 'OVK01717:   0.0006', 'OVK11693:   0.0006', 'OVK00974:   0.0006', 'OVK11686:   0.0010']

- 0.001 < min_Dist <=   0.1 |N:     41 (+    4) ( 73.2 %) | Example

The total number of segments that do not match is too high to check them all, but we'll check some cases. Check C:\OD\WS_Mdl\Mng\Mdl_Ipvs.xlsx/WBD_SHP_Eval for more info.

The ones <1m can be attributed to closing errors, and we can connect them to the closest one via an algorithm.<br>Let's print out the names of the bigger differences, so that we can check them in QGIS.

In [156]:
GDF.loc[GDF['min_Dist'].between(10,100, inclusive='right'), ['CODE', 'min_Dist']].sort_values(by='min_Dist')

,CODE,min_Dist
6851,OVK11551,38.279426
7472,OVK02333,56.202931
513,OVK02160,70.170908
493,OVK02830,70.170908
3815,OVK01792,84.207934


- **OVK11551**: -- TO BE CORRECTED -- is an anomaly, as it attaches to a very long feature, far from the final start and end.
- **OVK02333**: -- IGNORE -- ends outside the model, so there is no feature to connect to.
- **OVK02830** & **OVK02160**: -- IGNORE -- are in the same spot. one of them is straight, the other one circles around, and they both end outside the model.
- **OVK01792**: -- IGNORE -- also ends outside the model.

In [166]:
GDF.loc[GDF['min_Dist'].between(100,1000, inclusive='right'), ['CODE', 'min_Dist']].sort_values(by='min_Dist')

,CODE,min_Dist
3717,OVK02242,103.059615
8359,OVK02238,133.948685
491,OVK02121,158.327719
2474,OVK01807,192.357958
11549,OVK03078,227.105468
2792,OVK02887,360.424861
11029,OVK02933,364.087586
10250,OVK01682,428.826538
1913,OVK00416,564.173123


- **OVK02242**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK02238**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK02121**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK01807**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK03078**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK02887**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek (final Mark outled outside MdlAa) 
- **OVK02933**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK01682**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 
- **OVK00416**: --- IGNORE --- ends outside the model and doesn't have any impact on Chaamse Beek 


In [26]:
GDF[['CODE', 'min_Dist']].loc[GDF['toid'] == 0].sort_values(by='min_Dist', ascending=False)

,CODE,min_Dist
2903,OVK05520,2125.344220
10761,OVK10933,1243.853186
11424,OVK05519,1099.043328
10509,OVK03518,750.917601
9478,OVK20205,726.484193
...,...,...
2076,OVK01405,0.000100
12650,OVK03530,0.000100
12665,OVK08798,0.000100
12667,OVK05135,0.000100


In [27]:
print(f"Let's see how many features with min_Dist > 0.1 there are: {bold}{(GDF['min_Dist'] > 0.1).sum()}{style_reset} features, out of {GDF.loc[GDF['min_Dist']>0].shape[0]}. That's {round((GDF['min_Dist'] > 0.1).sum()/GDF.loc[GDF['min_Dist']>0].shape[0]*100, 2)} %. Which is quite high.")

Let's see how many features with min_Dist > 0.1 there are: 512 features, out of 1301. That's 39.35 %. Which is quite high.


Let's check some features where 1 < min_Dist < 10.

In [28]:
GDF[['CODE', 'min_Dist']].loc[(GDF['min_Dist'] < 10) & (GDF['min_Dist'] > 1)].sort_values(by='min_Dist', ascending=False)

,CODE,min_Dist
8384,OVK06539,9.976710
258,OVK08958,9.596619
4071,OVK08754,9.198640
6502,OVK12495,9.117206
5123,OVK12640,9.061852
...,...,...
5830,OVK10999,1.147236
8565,OVK10281,1.108484
6993,OVK10280,1.108484
9792,OVK12643,1.024820


In [ ]:
code, DStr = 'OKV21780', 'OVK04581'
print(f'')

In [ ]:
# Display the results
print("GDF with 'toid' column:")
GDF[['CODE', 'toid', 'Xstart', 'Ystart', 'Xend', 'Yend']].head()

In [ ]:
# Count the number of outlets (where toid is 0)
outlets = GDF[GDF['toid'] == 0]
print(f"\nNumber of outlets (no downstream connection): {bold}{len(outlets)}{style_reset}")

# Load and Explore **1ry SW Nw**

## Load

In [ ]:
# Read the hydroobject layer specifically
GDF = gpd.read_file(Pa_GPkg)

In [ ]:
print(f"\nHydroobject layer loaded successfully!\nNumber of features: {bold}{len(GDF)}{style_reset}\nCRS: {bold}{GDF.crs}{style_reset}\nGeometry types: {GDF.geometry.geom_type.value_counts()}")

## Explore

In [ ]:
GDF.describe(include='all')

In [ ]:
for i, col in enumerate(GDF.columns):
    print(f"{i}: {col}")

In [ ]:
GDF[['WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L', 'WS_BODEMBREEDTE_L', 'WS_TALUD_LINKS_L', 'WS_TALUD_RECHTS_L']].head(10)

## Input data requirements

Below, we'll match the shapefile attributes to the SFRmaker requirements as per SFRmaker manual ( https://doi-usgs.github.io/sfrmaker/latest/inputs.html):

**Custom hydrography** <br>
Any Polyline shapefile can be supplied in lieu of NHDPlus, but it must have the following columns, as shown in the second example:
1. flowlines_file: path to shapefile<br>-> Pa_GPkg_1ry (object, not column)
2. id_column: unique identifier for each polyline<br>-> CODE
3. routing_column: downstream connection (ID), 0 if none<br>->*Needs to be calculated based on algorithm*<br>
4. width1_column: channel width at start of line, in attr\_length\_units (optional)<br>-> WS_BH_BOVENSTROOMS_L
5. width2_column: channel width at end of line, in attr_length_units (optional)<br>->  WS_BH_BENEDENSTROOMS_L
6. up_elevation_column: streambed elevation at start of line, in attr_height_units<br>-> 
7. dn_elevation_column: streambed elevation at end of line, in attr_height_units<br>-> 
8. name_column: stream name (optional)<br>-> 
9. attr_length_units: channel width units<br>-> m
10. attr_height_units: streambed elevation units<br>-> m


Columns that seem useful:
1. CODE: as unique ID.
2. NAAM: as name.
3. WS_BH_BOVENSTROOMS: width upstream
4. WS_BH_BENEDENSTROOMS: width downstream
5. WS_HGT_MAAIVELD_BOVS: ground level elevation upstream
6. WS_HGT_MAAIVELD_BENS: ground level elevation downstream
7. WS_HGT_INSTEEK_LINKERZIJDE: left bank elevation
8. WS_HGT_INSTEEK_RECHTERZIJDE: right bank elevation
9. WS_TALUD_LINKS_L: left bank slope
10. WS_TALUD_RECHTS_L: right bank slope
11. WS_BODEMBREEDTE_L: bottom width
12. WS_BH_BOVENSTROOMS_L: width upstream (seems like a duplicate of 2, need to check though)
13. WS_BH_BENEDENSTROOMS_L: width downstream (seems like a duplicate of 3, need to check though)
14. 

In [ ]:
print('Plot of line geometry in the whole area.')
GDF.plot(figsize=(10, 10), color='blue', edgecolor='black')

## Limit

In [ ]:
# Limit GDF to model area bounds
print(f"Model area bounds: Xmin: {Xmin:10}, Xmax: {Xmax:10} | Ymin: {Ymin:10}, Ymax: {Ymax:10}\n")
print(f"Original dataset: {bold}{len(GDF):10}{style_reset} features\t| Bounds: {GDF.total_bounds}")

# Create bounding box for clipping
from shapely.geometry import box
model_bbox = box(Xmin, Ymin, Xmax, Ymax)

# Filter GDF to model area using spatial intersection
GDF_clipped = GDF[GDF.geometry.intersects(model_bbox)].copy()

print(f"Filtered dataset: {bold}{len(GDF_clipped):10}{style_reset} features\t| Bounds: {GDF_clipped.total_bounds}")
print(f"Reduction: {bold}{len(GDF)-len(GDF_clipped):,}{style_reset} features removed ({(len(GDF)-len(GDF_clipped))/len(GDF)*100:.1f}%) | ✓ GDF updated to model area extent")
print(f"✓ Network length: {GDF.geometry.length.sum()/1000:.1f} km within model area")

# Update GDF to use the clipped version
GDF = GDF_clipped

In [ ]:
# Plot the filtered hydroobject network within model area
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the filtered GDF
GDF.plot(ax=ax, color='blue', linewidth=0.8, alpha=0.7, label='Hydroobject network')

# Add model boundary rectangle
from matplotlib.patches import Rectangle
model_rect = Rectangle((Xmin, Ymin), Xmax-Xmin, Ymax-Ymin, 
                      linewidth=2, edgecolor='red', facecolor='none', 
                      linestyle='--', label='Model boundary')
ax.add_patch(model_rect)

ax.set_title(f'Hydroobject Network within Model Area ({MdlN})')
ax.set_xlabel('X coordinate (m)')
ax.set_ylabel('Y coordinate (m)')
ax.legend()

# Add statistics text
stats_text = f"""Filtered Network:
• Features: {len(GDF):,}
• Total length: {GDF.geometry.length.sum()/1000:.1f} km
• Model area: {(Xmax-Xmin)/1000:.1f} × {(Ymax-Ymin)/1000:.1f} km"""

ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Add grid
ax.grid(True, alpha=0.3)

# Create Grid and Generate SFR Routing Network

In [ ]:
help(sfr.Lines.from_dataframe)

In [ ]:
GDF.columns

In [ ]:
lines = sfr.Lines.from_dataframe(
    GDF,
    id_column='CODE',
    routing_column='toid',                      # <- add this (see quick builder below)
    width1_column='W', width2_column='W',
    up_elevation_column='WS_BH_BOVENSTROOMS_L',
    dn_elevation_column='WS_BH_BENEDENSTROOMS_L',
    name_column='LEGGER_KENMERK',
    geometry_column=GDF.geometry.name,
)


# Junkyard

In [ ]:
stop

# Load GPkg. Select correct layer. Limit to Mdl Aa. - Detailed SW network

After this methodology is established, a function should be created that'll take in either a GeoPackage with a specified layer or a shapefile (single layer).<br>
Other Params like the Cols to be used for the SFR network creation, should be provided as well. A .toml file could be a good idea.

## Load

In [ ]:
# First, let's see what layers are available in the GeoPackage
layers = fiona.listlayers(Pa_GPkg)
print("Available layers in GeoPackage:")
for i, L in enumerate(layers):
    print(f"  {i+1}. {L}")

In [ ]:
# Read the hydroobject layer specifically
GDF1 = gpd.read_file(Pa_GPkg, layer=layer)

In [ ]:
print(f"\nHydroobject layer loaded successfully!\nNumber of features: {bold}{len(GDF1)}{style_reset}\nCRS: {bold}{GDF1.crs}{style_reset}\nGeometry types: {GDF1.geometry.geom_type.value_counts()}")
print(*GDF1.columns, sep=' | ')

## Explore

In [ ]:
GDF1.head(3)

In [ ]:
GDF1.describe(include='all')

In [ ]:
#666 columns need to be reviewed later!!!

In [ ]:
GDF1.WS_BH_BOVENSTROOMS.value_counts()

In [ ]:
for i, col in enumerate(GDF1.columns):
    print(f"{i}: {col}")

In [ ]:
GDF1.sort_values('WS_BH_BOVENSTROOMS', inplace=True, ascending=False)

In [ ]:
GDF1[['WS_BH_BOVENSTROOMS', 'WS_BH_BENEDENSTROOMS', 'WS_BH_BOVENSTROOMS_L', 'WS_BH_BENEDENSTROOMS_L']].head(10)

Columns that seem useful:
1. CODE: as unique ID.
2. NAAM: as name.
3. WS_BH_BOVENSTROOMS: width upstream
4. WS_BH_BENEDENSTROOMS: width downstream
5. WS_HGT_MAAIVELD_BOVS: ground level elevation upstream
6. WS_HGT_MAAIVELD_BENS: ground level elevation downstream
7. WS_HGT_INSTEEK_LINKERZIJDE: left bank elevation
8. WS_HGT_INSTEEK_RECHTERZIJDE: right bank elevation
9. WS_TALUD_LINKS_L: left bank slope
10. WS_TALUD_RECHTS_L: right bank slope
11. WS_BODEMBREEDTE_L: bottom width
12. WS_BH_BOVENSTROOMS_L: width upstream (seems like a duplicate of 2, need to check though)
13. WS_BH_BENEDENSTROOMS_L: width downstream (seems like a duplicate of 3, need to check though)
14. 

In [ ]:
print('Plot of line geometry in the whole area.')
GDF1.plot(figsize=(10, 10), color='blue', edgecolor='black')

## Limit

In [ ]:
# Limit GDF to model area bounds
print(f"Model area bounds: Xmin: {Xmin:10}, Xmax: {Xmax:10} | Ymin: {Ymin:10}, Ymax: {Ymax:10}\n")
print(f"Original dataset: {bold}{len(GDF1):10}{style_reset} features\t| Bounds: {GDF1.total_bounds}")

# Create bounding box for clipping
from shapely.geometry import box
model_bbox = box(Xmin, Ymin, Xmax, Ymax)

# Filter GDF to model area using spatial intersection
GDF1_clipped = GDF1[GDF1.geometry.intersects(model_bbox)].copy()

print(f"Filtered dataset: {bold}{len(GDF1_clipped):10}{style_reset} features\t| Bounds: {GDF1_clipped.total_bounds}")
print(f"Reduction: {bold}{len(GDF1)-len(GDF1_clipped):,}{style_reset} features removed ({(len(GDF1)-len(GDF1_clipped))/len(GDF1)*100:.1f}%) | ✓ GDF updated to model area extent")
print(f"✓ Network length: {GDF1.geometry.length.sum()/1000:.1f} km within model area")

# Update GDF to use the clipped version
GDF1 = GDF1_clipped

In [ ]:
# Plot the filtered hydroobject network within model area
fig, ax = plt.subplots(figsize=(12, 8))

# Plot the filtered GDF
GDF1.plot(ax=ax, color='blue', linewidth=0.8, alpha=0.7, label='Hydroobject network')

# Add model boundary rectangle
from matplotlib.patches import Rectangle
model_rect = Rectangle((Xmin, Ymin), Xmax-Xmin, Ymax-Ymin, 
                      linewidth=2, edgecolor='red', facecolor='none', 
                      linestyle='--', label='Model boundary')
ax.add_patch(model_rect)

ax.set_title(f'Hydroobject Network within Model Area ({MdlN})')
ax.set_xlabel('X coordinate (m)')
ax.set_ylabel('Y coordinate (m)')
ax.legend()

# Add statistics text
stats_text = f"""Filtered Network:
• Features: {len(GDF1):,}
• Total length: {GDF1.geometry.length.sum()/1000:.1f} km
• Model area: {(Xmax-Xmin)/1000:.1f} × {(Ymax-Ymin)/1000:.1f} km"""

ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, 
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

# Add grid
ax.grid(True, alpha=0.3)